In [1]:
# Reading/Writing Data
import os
import glob
import numpy as np
import math 

# Pytorch
import torch 
from torch.utils.data import Subset, DataLoader

# Self-Defined Package
from SMAPDataset import SMAPDataset
from SMCNN import SMCNN
from SDLoss import physics_loss
from Preprocessing import random_spatial_sequence, collate_fn

# Some Utility Functions

In [2]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'test_ratio': 0,
    'root': 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\'
}

# Dataloader
Read data from files and set up training, validation, and testing sets.

In [4]:
# # Set seed for reproducibility
same_seed(config['seed'])

In [5]:
train_sequence = {}
test_sequence = {}
s2s_dir = config['root'] + 'LABEL\\SMAPID2INSITUID'
subdir_list = sorted(os.listdir(s2s_dir))

# 遍历指定路径下的所有子目录
for subdir in subdir_list:
    subdir_path = os.path.join(s2s_dir, subdir)
    if os.path.isdir(subdir_path):
        # 如果子目录是目录而不是文件，则将子目录加入字典
        full_spatial_sequence_smap = sorted([int(f.split('.')[0]) for f in os.listdir(subdir_path) if f.endswith('.npy')]) # !!!! read out of order
        print(len(full_spatial_sequence_smap), 'of Full Spatial Sequence for', subdir, ':')
        print(full_spatial_sequence_smap)
        
        # Split the train\valid\test dataset by spatial dimension
        train_sequence[subdir], test_sequence[subdir] = random_spatial_sequence(1-config['test_ratio'], full_spatial_sequence_smap)

10 of Full Spatial Sequence for 2015104 :
[9, 15, 17, 18, 20, 26, 29, 31, 40, 43]
**************************Data Spliting***************************
Spliting Rate:  1
10 of Dataset1:  [15 31 18 40 29 17 43 26 20  9]
0 of Dataset2:  []
**************************Data Spliting***************************
6 of Full Spatial Sequence for 2015106 :
[15, 16, 20, 26, 27, 29]
**************************Data Spliting***************************
Spliting Rate:  1
6 of Dataset1:  [26 20 27 29 15 16]
0 of Dataset2:  []
**************************Data Spliting***************************
4 of Full Spatial Sequence for 2015114 :
[9, 33, 44, 51]
**************************Data Spliting***************************
Spliting Rate:  1
4 of Dataset1:  [33 51  9 44]
0 of Dataset2:  []
**************************Data Spliting***************************
14 of Full Spatial Sequence for 2015115 :
[15, 16, 17, 18, 20, 26, 29, 31, 32, 33, 40, 41, 43, 44]
**************************Data Spliting***************************
S

In [29]:
print(full_spatial_sequence_smap)

[9, 10, 15, 16, 17, 18, 20, 26, 27, 29, 31, 32, 33, 40, 41, 43, 44, 45, 51]


In [6]:
# Initialize the dataset
train_dataset = SMAPDataset(config['root'], train_sequence)

***************************Load data path******************************
_______________________________2015104_______________________________
_____________________________smap cell: 15_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015104\15.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\15.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015104\4.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015104\4.npy
_____________________________smap cell: 31_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015104\31.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\31.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015104\10.npy
D:\1GRADUATED\paper\downscaling_data\S

D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015200\12.npy
_____________________________smap cell: 16_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015200\16.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\16.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015200\1.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015200\1.npy
_____________________________smap cell: 15_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015200\15.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\15.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015200\4.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\

D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015232\11.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015232\11.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015232\42.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015232\42.npy
_____________________________smap cell: 43_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015232\43.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\43.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015232\53.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015232\53.npy
_____________________________smap cell: 40_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT

In [7]:
train_dataset.__len__()

409

In [8]:
process_data = [x['processed_data'] for x in train_dataset]
label_data = [x['label_data'] for x in train_dataset]
meta_data = [x['meta_data'] for x in train_dataset]

# Dataset Checking Manually

In [41]:
date = [x['date'] for x in meta_data]
smapid = [x['smapid'] for x in meta_data]
insituid = [x['insituid'] for x in meta_data]

In [106]:
import random

start = 1
end = 409
num_samples = 1

samples = np.random.randint(start, end+1, size=num_samples)
print(samples)

[203]


In [107]:
import datetime

# The index of day start from 1
# day 从 1 开始
def date2day(date, fm="%Y-%m-%d"):
    date_object = datetime.datetime.strptime(date, fm)
    day_of_year = date_object.timetuple().tm_yday
    return day_of_year

def day2date(year, day):
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(day - 1)
    month = date.month
    day = date.day
    return date

In [108]:
def calculate_standard_deviation(theta_ij, theta_bar, ati, ati_mean, ati_sd):
    return (theta_ij-theta_bar)/(ati-ati_mean)*ati_sd

In [109]:
for i in samples:
    day = date[i].split('2015')[-1]
    year = date[i].split(day)[0]
    _date = day2date(int(year), int(day)).strftime("%Y-%m-%d")
    print('日期：{} {}'.format(_date, date[i]))
    print('SMAPID：{}'.format(smapid[i]))
    print('smap_norm：{}'.format([process_data[i][0][0][0]]))
    print('len of list：{} {}'.format(len(insituid[i]), len(label_data[i])))
    for j in range(len(insituid[i])):
        sd = calculate_standard_deviation(label_data[i][j][0], label_data[i][j][1], label_data[i][j][2][0], label_data[i][j][2][1], label_data[i][j][2][2])
        print('站点ID：{}'.format(insituid[i][j]))
        print('sm：{}'.format(label_data[i][j][0]))
        print('smap_unorm：{}'.format(label_data[i][j][1]))
        print('ati, atim, atisd：{}'.format(label_data[i][j][2]))
        print('sd: {}'.format(sd))

日期：2015-08-14 2015226
SMAPID：27
smap_norm：[1.1395782232284546]
len of list：1 1
站点ID：12
sm：[0.42632609]
smap_unorm：[0.3225762]
ati, atim, atisd：[0.0689095028062 0.057638898491859436 0.005128140095621347]
sd: [0.04720634]


In [50]:
print(date2day('2015-10-01'))

274


In [56]:
root = 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\LABEL\\SM\\2015274\\'

In [57]:
sm = np.load(root+'12.npy')
print(sm)

[0.44075972]


In [58]:
root = 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\LABEL\\SMAP\\2015274\\'

In [59]:
smap = np.load(root+'27.npy')
print(smap)

[0.2718554]


In [60]:
root = 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\LABEL\\ATI\\2015274\\'

In [61]:
arr = np.load(root+'12.npy', allow_pickle=True)
print(arr)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\LABEL\\ATI\\2015274\\12.npy'

In [17]:
sd = calculate_standard_deviation

2015-04-14 00:00:00


## 经过检验：
> - dataset读取的数据与文件夹里的相同
> - 文件夹里的数据（smap，ati）与arcmap对齐后提取的excel里的数据相同
> - 提取的excel与原始tiff相同

## 检查smap与texture是否对齐

In [18]:
x = 2
y = 2
for i in samples:
    print(process_data[i][2, 2, 0])
    print(process_data[i][2, 2, 1])
    print(process_data[i][2, 2, 2])

0.41653692722320557
13.0
56.0


# Visualize Calculated SD

In [19]:
def calculate_standard_deviation(theta_ij, theta_bar, ati, ati_mean, ati_sd):
    return (theta_ij-theta_bar)/(ati-ati_mean)*ati_sd

In [28]:
sd_list = []
for data_list in label_data:
    sd_i_list = []
    for data in data_list:
        sd = calculate_standard_deviation(data[0][0], data[1][0], data[2][0], data[2][1], data[2][2])
        sd_i_list.append(sd)
    sd_list.append(sd_i_list)
    print(sd_i_list)
# print([x for x in sd_list if len(x)>1])

[-0.09044444477656413]
[-0.787583344986823]
[0.0950151604154732]
[0.5064270294078725]
[-0.10347082732939006]
[-0.15388406484790917]
[0.11240704132615417]
[0.4983206177410777, -0.10330019790887278]
[-0.42621344090384894]
[0.40377774950503215]
[-0.21132337388658848, -0.3002622892688309]
[0.2124573297134763]
[1.1275235483039596]
[0.1994607780108491]
[-0.030955626466968002]
[-0.5814780581648974]
[0.035402763564983016]
[0.17269832274805116]
[-0.08495636760999513]
[-0.24861339296287738]
[0.24919149072436933]
[0.07126106418545025]
[-0.0011445969677155437]
[0.1672299524287562, 0.07395230796067619]
[-0.14319119701358696]
[-0.11014721733876874]
[-0.009271672672946488]
[0.04364230455413831, -0.029697060103123698]
[0.14683327782456265]
[0.20629198212575617]
[0.6028229541991049]
[-0.09023804748336076, -0.059303312848717524]
[0.005495869114773241]
[0.015885246436391758]
[-0.04637033789395278]
[0.06130556632882372]
[-1.792035876541754]
[0.4716772990745133]
[0.00698066626379233]
[0.7637601405175838]
[

In [30]:
print(int(409*0.98))

400
